## Query Marketplace

This notebook constructs a "query marketplace", where our system selects the fingerprints with the highest utility to cost ratio at runtime and does lookups only on those fingerprints.

In [ ]:
import pickle
import numpy as np
from ExtractBootlegScores import *
import itertools
import numba as nb
from numba import njit
from collections import defaultdict
import dill
from glob import iglob

**Load in databases and counts**

In [ ]:
# this dictionary contains all the databases for each n-gram type included in our marketplace database
db_dir = "/data1/kji/databases_v4d/105mill"

In [ ]:
# this dictionary maps each n-gram type to its probability of correctness
probabilities_dir = "/data1/kji/databases_random/probabilities.pkl"

In [ ]:
# this dictionary maps each number to the piece name (a string)
piece_mapping_dir = "num_to_piece.pkl"

In [ ]:
d = {}

In [ ]:
for filename in iglob(f"{db_dir}/*.pkl", recursive=True):
    combination = filename.split('/')[-1][:-4]
    with open(filename, "rb") as f:
        d[combination] = pickle.load(f)
    print(f"finished {combination}")

In [ ]:
with open(probabilities_dir, "rb") as f:
    utilities = pickle.load(f)

In [ ]:
with open(piece_mapping_dir, 'rb') as f:
    num_to_piece = pickle.load(f)

In [ ]:
combinations = []
for n_gram in range(1, 4):
    combinations += [[0] + list(tup) for tup in itertools.combinations(range(1, 7), n_gram-1)]

In [ ]:
combinations = ["".join(str(num) for num in combination) for combination in combinations]

In [ ]:
powers = 1 << np.arange(62)[::-1]

In [ ]:
def compute_fingerprint(cols):
    fp = []
    equals_Zero = True
    for column in cols:
        hashint = int(column.dot(powers))
        fp.append(hashint)
        if hashint != 0:
            equals_Zero = False
    if equals_Zero == True:
        return None
    return tuple(fp)

Our utility to cost ratio is defined as the probability of correctness (utility) divided by the number of matches (cost), because the number of matches equals the number of lookups we would have to do for that fingerprint.

In [ ]:
def utility(combination, matches):
    return utilities[combination] / matches

In [ ]:
def get_ratios(bscore_query, rindex_dict):
    """Inputs: an L x 62 bootleg score query and our dictionary, where
               rindex_dict[fp] = (count, {dictionary of pieces and offsets})
        Output: a 16 X L table where each element is a tuple of (utility:cost ratio, combination, n_gram)"""
    l = len(bscore_query)
    # ratios[i][j] is a pair of (ratio, combination, fingerprint)
    ratios = np.array([[(0, None, None) for _ in range(l)] for _ in range(16)])
    for j in range(l):
        # calculate utility to cost ratio for all 16 n-grams
        for idx, combination in enumerate(combinations):
            cols = []
            # we need at least enough fingerprints for all the indices in our combination
            try:
                for i in combination:
                    cols.append(bscore_query[j+int(i)])
            except IndexError:
                continue
            fp = compute_fingerprint(cols)
            if not fp or combination not in rindex_dict or fp not in rindex_dict[combination]:
                continue
            matches = rindex_dict[combination][fp][0]
            if matches == 0:
                continue
            ratios[idx][j] = (utility(combination, matches), combination, fp)
    return ratios

In [ ]:
def update_offset_dict(offset_dict, pieces_and_offsets, i, num_lookups):
    """Input: a dictionary mapping pieces to offsets for a given n-gram and the number of lookups we can do.
       Output: an updated dictionary of offsets containing all the lookups we just did. This dictionary
               will be used in the histogram of offsets."""
    if num_lookups == 0:
        return
    for piece in pieces_and_offsets:
        if num_lookups <= len(pieces_and_offsets[piece]):
            offset = [j - i for j in pieces_and_offsets[piece][:num_lookups]]
        else:
            offset = [j - i for j in pieces_and_offsets[piece]]
        offset_dict[num_to_piece[piece]].extend(offset)
        num_lookups -= len(pieces_and_offsets[piece])
        if num_lookups <= 0:
            break

In [ ]:
def get_fingerprints(bscore_query, rindex_dict, ratios, runtime_budget):
    """This takes in a bootleg score for a query as its input, and does a certain number of lookups for each
       column of the bootleg score in accordance with our runtime budget. It then returns the updated dictionary of
       offsets containing all the lookups performed for the query, which is then used to compute the histogram of 
       offsets."""
    l = len(bscore_query)
    aisle_budget = runtime_budget // l
    cur_budget = aisle_budget
    offset_dict = defaultdict(list)
    matches_processed = 0
    for i in range(l):
        fingerprints = []
        col = ratios[:, i]
        lookups = sorted(col, key = lambda x: x[0], reverse = True)
        for _, combination, n_gram in lookups:
            if not n_gram or cur_budget < 0:
                break
            matches, pieces_and_offsets = rindex_dict[combination][n_gram]
            if cur_budget - matches < 0:
                num_lookups = cur_budget
            else:
                num_lookups = matches
            update_offset_dict(offset_dict, pieces_and_offsets, i, num_lookups)
            cur_budget -= num_lookups
            matches_processed += num_lookups
        cur_budget += aisle_budget
    return offset_dict, matches_processed

In [ ]:
def rankHistograms(offset_dict, bin_size=10):
    """This implements the histogram of offsets method for ranking the predicted pieces."""
    bin_size = 3
    pieceScores = []
    for key, h in offset_dict.items():
        if not h:
            continue
        maxh = max(h)
        minh = min(h)
        hist = np.zeros(int((maxh-minh)/bin_size)+2)
        for i in h:
            hist[int((i-minh)/bin_size)] += 1
        score = np.max(hist)
        pieceScores.append((key, score))
            
    pieceScores = sorted(pieceScores, key = lambda x:x[1], reverse=True)
    return pieceScores

In [ ]:
def processSingleQuery(imagefile, rindex, runtime_budget, outfile = None):
    """Inputs: a file representing a query image, a reverse index dictionary mapping each n-gram to its 
               offsets in IMSLP, and a runtime budget for the query.
       Output: a sorted list of predicted pieces and their scores based on the histogram of offsets method."""
    profileStart = time.time()
    
    # Get Bootleg Score
    bscore_query = processQuery(imagefile)
    bscore_query = bscore_query.T
    
    searchStart = time.time()
    # Generate and rank histograms
    
    ratios = get_ratios(bscore_query, rindex)
    offset_dict, matches_processed = get_fingerprints(bscore_query, rindex, ratios, runtime_budget)
    pieceScores = rankHistograms(offset_dict)
    # Profile & save to file
    profileEnd = time.time()
    
    profileDur = profileEnd - profileStart
    print(matches_processed)
    print(profileDur)
    saveToFile(outfile, imagefile, pieceScores, profileDur, matches_processed)
    return pieceScores

In [ ]:
processSingleQuery('data/queries/p146_q2.jpg', d, 75000)

In [ ]:
def saveToFile(outfile, imagefile, pieceScores, profileDur, matches_processed):
    if outfile:
        with open(outfile, 'wb') as f:
            query = os.path.splitext(os.path.basename(imagefile))[0]
            pickle.dump((query,pieceScores, profileDur, matches_processed),f)

In [ ]:
def processQuery_wrapper(queryfile, rindex, outdir, runtime_budget):
    # wrapper for running multiple jobs in parallel
    basename = os.path.splitext(os.path.basename(queryfile))[0] # e.g. p1_q1
    hyp_outfile = "{}/{}.hyp".format(outdir, basename)
    piece = basename.split('_')[0]
    # might change later to print to outfile
    return processSingleQuery(queryfile, rindex, runtime_budget, hyp_outfile)

Here we set the runtime budget to 65000, which means we can process at most 65000 matches per query.

In [ ]:
runtime_budget = 65000

In [ ]:
query_list = 'cfg_files/query.test.list' # list of query images
outdir = f'experiments/v0.4.0d_test_65k_budget/hyp' # where to save hypothesis output files

In [ ]:
# prep output directory
if not os.path.isdir(outdir):
    os.makedirs(outdir)

# load reverse index. Recommend keeping load=False and loading it earlier.
load = False
if load:
    print("LOADING RINDEX")
    rindex1 = []
    with open(pickle_file, 'rb') as f:
        rindex1 = pickle.load(f)
    rindex_filter = rindex1

print("STARTING PROCESSING")
# number of cores to use
multiprocess = False
if multiprocess:
    n_cores = 30 #multiprocessing.cpu_count()
    pool = multiprocessing.Pool(processes=n_cores)

inputs = []
with open(query_list, 'r') as f:
    for line in f:
        inputs.append((line.rstrip(), outdir))

if multiprocess:
    # process queries in parallel
    outputs = list(pool.starmap(processQuery_wrapper, inputs))
else:
    for i in inputs:
        processQuery_wrapper(i[0], d, i[1], runtime_budget)